In [ ]:
# https://huggingface.co/blog/how-to-train

In [34]:
!pip install datasets
!pip install nltk

In [35]:
import transformers
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from tokenizers import ByteLevelBPETokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaTokenizerFast
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import SequentialSampler
from datasets import load_dataset
import os
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
ds = load_dataset("abisee/cnn_dailymail", "1.0.0")

In [38]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [39]:
ds["train"][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [40]:
#the news article is in the article key
ds["train"][4]["article"]

'(CNN)  -- The National Football League has indefinitely suspended Atlanta Falcons quarterback Michael Vick without pay, officials with the league said Friday. NFL star Michael Vick is set to appear in court Monday. A judge will have the final say on a plea deal. Earlier, Vick admitted to participating in a dogfighting ring as part of a plea agreement with federal prosecutors in Virginia. "Your admitted conduct was not only illegal, but also cruel and reprehensible. Your team, the NFL, and NFL fans have all been hurt by your actions," NFL Commissioner Roger Goodell said in a letter to Vick. Goodell said he would review the status of the suspension after the legal proceedings are over. In papers filed Friday with a federal court in Virginia, Vick also admitted that he and two co-conspirators killed dogs that did not fight well. Falcons owner Arthur Blank said Vick\'s admissions describe actions that are "incomprehensible and unacceptable." The suspension makes "a strong statement that c

In [41]:
df = pd.DataFrame(ds["train"])

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
 2   id          287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [43]:
df.isnull().sum()

,0
article,0
highlights,0
id,0


In [44]:
#reducing the size of the dataset since the total would be way too big for this BabyLM
df = df[:10000]

In [45]:
# Separating sentences
df_sent = df
df_sent['sentences'] = [nltk.sent_tokenize(text) for text in df_sent['article']]

In [46]:
# Making a single list of all the sentences in each article
sentences = sum(df_sent['sentences'].tolist(), [])

In [47]:
def preprocess(text, punctuation_marks):
    """Remove the punctuation specified by the user, and return the same string without the unwanted characters.

    Parameters:
    - 'text': string to process and remove characters from.
    - 'punctuation marks': characters to be removed, given in list format. It can be any number of characters."""

    for character in punctuation_marks: #iterating over the characters given in the parameter list 'punctuation_marks'
        text = text.replace(character, '') #the characters given will be removed from the string by replacing them with empty strings ''
    preprocessesed_text = text #assigning the variable text to a new variable so that it returns the text with all the instances of the punctuation marks removed.
    return preprocessesed_text

In [48]:
cleaned_sentences = []
for sent in sentences:
  cleaned = preprocess(sent,['"']) #removing quotation marks so that the text is cleaner for the model
  cleaned_sentences.append(cleaned)

In [49]:
print(cleaned_sentences[:5])

["LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.", 'Daniel Radcliffe as Harry Potter in Harry Potter and the Order of the Phoenix To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.', "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar, he told an Australian interviewer earlier this month.", "I don't think I'll be particularly extravagant.", 'The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs.']


In [50]:
# Tokenizing each sentence in cleaned_sentences
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in cleaned_sentences]

# Calculate the total number of tokens
total_tokens = sum(len(tokens) for tokens in tokenized_sentences)

print(f"Total number of tokens: {total_tokens}")

Total number of tokens: 7061584


In [51]:
sequences = cleaned_sentences

In [ ]:
##Celonie's code starts here

#train_filepath = 'data/ranked_sequences.csv'


#ranked = pd.read_csv(train_filepath)
#sequences = ranked['sequence'].tolist()

#with open('combined.dev') as infile:
#  devset = infile.read().split("\n")[:10000]


In [52]:
len(sequences)

329883

In [53]:
#split data into train and dev

df_train, df_dev = train_test_split(sequences, test_size=0.15, random_state=None)

In [54]:
len(df_train)

280400

In [56]:
## got until here 20/Sep

In [ ]:
tokenizer_folder = 'tokenizer_folder'

if not os.path.exists(tokenizer_folder):
    os.mkdir(tokenizer_folder)

In [ ]:

# Initialize tokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train_from_iterator(df_train, vocab_size=52_000, min_frequency=2, show_progress=True, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save tokenizer
tokenizer.save_model(tokenizer_folder)

In [ ]:

# Configuration for RoBERTa model
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize model
model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

In [ ]:
# tokenizer_folder = 'tokenizer_folder'

# Load tokenizer
max_length = 512
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=max_length)

In [ ]:
# create CustomDataset class

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.examples = []
        self.mask = []

        for example in data:
            x=tokenizer.encode_plus(example, max_length = max_length, truncation=True, padding=True)
            self.examples += [x.input_ids]
            self.mask += [x.attention_mask]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i])

# create train and evaluation datasets
train_dataset = CustomDataset(df_train, tokenizer)
eval_dataset = CustomDataset(df_dev, tokenizer)

In [ ]:

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
#adapted from https://github.com/MiuLab/FastMTL/blob/only3/custom_trainer.py

# adapt get_train_dataloader function to supply DataLoader using SequentialSampler and shuffle=False to enforce curriculum learning

from transformers.trainer import *
def get_train_dataloader(self) -> DataLoader:
    """
    Returns the training :class:`~torch.utils.data.DataLoader`.

    Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
    to distributed training if necessary) otherwise.

    Subclass and override this method if you want to inject some custom behavior.
    """
    if self.train_dataset is None:
        raise ValueError("Trainer: training requires a train_dataset.")

    return DataLoader(
        self.train_dataset,
        batch_size=self.args.train_batch_size,
        sampler=SequentialSampler(self.train_dataset),
        collate_fn=self.data_collator,
        drop_last=self.args.dataloader_drop_last,
        num_workers=self.args.dataloader_num_workers,
        shuffle=False
    )
Trainer.get_train_dataloader = get_train_dataloader

In [ ]:
batch_size = 16

# Define training arguments
training_args = TrainingArguments(
    output_dir='model_folder',
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=5,
    learning_rate=1e-4,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=8192,
    save_total_limit=1,
    #seed=10098,
    max_steps=int(1269227 / batch_size)
)
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
# Train the model
trainer.train()